In [1]:
import sys
sys.path.insert(0, '../Core-scripts/')

from parse_and_prepare import ProteinProteinInteractionClassifier as ppi
import file_readers as fr
import prediction as pred

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import pickle
import os
import re
from sklearn.metrics import roc_curve, auc
import numpy as np
from gensim.models import word2vec
import logging

In [3]:
dros_strict_real = pickle.load(open('../../Results/Drosophila/drosophila_mentions_strict_real.pkl', 'rb'))
dros_gen_real = pickle.load(open('../../Results/Drosophila/drosophila_mentions_gen_real.pkl', 'rb'))
dros_be_real = pickle.load(open('../../Results/Drosophila/drosophila_mentions_be_real.pkl', 'rb'))
random_seeds = [144, 235, 905, 2895, 3462, 4225, 5056, 5192, 7751, 7813]

In [6]:
for seed in random_seeds:
    real_tr_te_name = 'Drosophila/train_test/dros_tr_te_split_' + str(seed)
    train_data, b, c, d = pred.manual_train_test_split(dros_strict_real, real_tr_te_name, random_state=seed ,test_set_prop=0.1)
    tr_val_name = 'Drosophila/train_val/dros_tr_val_split_' + str(seed)
    real_train_data, b, c, d = pred.manual_train_test_split(train_data, tr_val_name, random_state=seed, test_set_prop=0.2)

In [10]:
dimensions = [100, 200, 300, 400, 500, 600]
min_word_count = [1, 2, 3, 4, 5, 6]
context_window = [2, 3, 4, 5, 6, 7]
downsample = [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001]

In [4]:
dimensions = [100, 200, 300, 400, 500, 600]
for i in dimensions:
    w2v_parameters = [i, 5, 4, 6, 0.001]
    strict = pred.make_w2v_model(dros_strict_real, 'Drosophila/models/dims/dros_strict_dim_parameter_' + str(i), w2v_parameters)
    gen = pred.make_w2v_model(dros_gen_real, 'Drosophila/models/dims/dros_gen_dim_parameter_' + str(i), w2v_parameters)
    be = pred.make_w2v_model(dros_be_real, 'Drosophila/models/dims/dros_be_dim_parameter_' + str(i), w2v_parameters)

2017-05-12 18:01:36,501 : INFO : collecting all words and their counts
2017-05-12 18:01:36,502 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-12 18:01:36,544 : INFO : collected 12999 word types from a corpus of 188627 raw words and 7408 sentences
2017-05-12 18:01:36,545 : INFO : Loading a fresh vocabulary
2017-05-12 18:01:36,557 : INFO : min_count=5 retains 3553 unique words (27% of original 12999, drops 9446)
2017-05-12 18:01:36,559 : INFO : min_count=5 leaves 173308 word corpus (91% of original 188627, drops 15319)
2017-05-12 18:01:36,567 : INFO : deleting the raw counts dictionary of 12999 items
2017-05-12 18:01:36,570 : INFO : sample=0.001 downsamples 35 most-common words
2017-05-12 18:01:36,571 : INFO : downsampling leaves estimated 124688 word corpus (71.9% of prior 173308)
2017-05-12 18:01:36,572 : INFO : estimated required memory for 3553 words and 100 dimensions: 4618900 bytes
2017-05-12 18:01:36,586 : INFO : resetting layer weights
2017-05-

Parsing datasets sentences
Training Word2Vec Model


2017-05-12 18:01:37,244 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-05-12 18:01:37,249 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-05-12 18:01:37,250 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-05-12 18:01:37,257 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-05-12 18:01:37,259 : INFO : training on 943135 raw words (623249 effective words) took 0.6s, 994418 effective words/s
2017-05-12 18:01:37,260 : INFO : precomputing L2-norms of word weight vectors
2017-05-12 18:01:37,283 : INFO : saving Word2Vec object under ../../Results/Drosophila/models/dims/dros_strict_dim_parameter_100_model, separately None
2017-05-12 18:01:37,284 : INFO : not storing attribute syn0norm
2017-05-12 18:01:37,285 : INFO : not storing attribute cum_table
2017-05-12 18:01:37,322 : INFO : saved ../../Results/Drosophila/models/dims/dros_strict_dim_parameter_100_model


Parsing datasets sentences


2017-05-12 18:01:37,836 : INFO : collecting all words and their counts
2017-05-12 18:01:37,837 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-12 18:01:37,898 : INFO : PROGRESS: at sentence #10000, processed 240061 words, keeping 13294 word types
2017-05-12 18:01:37,961 : INFO : PROGRESS: at sentence #20000, processed 479451 words, keeping 17608 word types
2017-05-12 18:01:38,028 : INFO : PROGRESS: at sentence #30000, processed 719251 words, keeping 20837 word types


...

2017-05-12 18:06:26,036 : INFO : collected 22824 word types from a corpus of 904687 raw words and 37697 sentences
2017-05-12 18:06:26,037 : INFO : Loading a fresh vocabulary
2017-05-12 18:06:26,076 : INFO : min_count=5 retains 8096 unique words (35% of original 22824, drops 14728)
2017-05-12 18:06:26,077 : INFO : min_count=5 leaves 879988 word corpus (97% of original 904687, drops 24699)
2017-05-12 18:06:26,102 : INFO : deleting the raw counts dictionary of 22824 items
2017-05-12 18:06:26,104 : INFO : sample=0.001 downsamples 40 most-common words
2017-05-12 18:06:26,110 : INFO : downsampling leaves estimated 644974 word corpus (73.3% of prior 879988)
2017-05-12 18:06:26,111 : INFO : estimated required memory for 8096 words and 600 dimensions: 42908800 bytes
2017-05-12 18:06:26,153 : INFO : resetting layer weights
2017-05-12 18:06:26,310 : INFO : training model with 4 workers on 8096 vocabulary and 600 features, using sg=0 hs=0 sample=0.001 negative=5 window=6
2017-05-12 18:06:26,311 : 

Parsing datasets sentences


2017-05-12 18:06:45,496 : INFO : collecting all words and their counts
2017-05-12 18:06:45,497 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-12 18:06:45,563 : INFO : PROGRESS: at sentence #10000, processed 223978 words, keeping 13703 word types
2017-05-12 18:06:45,637 : INFO : PROGRESS: at sentence #20000, processed 448086 words, keeping 18835 word types


Training Word2Vec Model


2017-05-12 18:06:45,720 : INFO : PROGRESS: at sentence #30000, processed 672056 words, keeping 22546 word types
2017-05-12 18:06:45,823 : INFO : PROGRESS: at sentence #40000, processed 896507 words, keeping 25560 word types
2017-05-12 18:06:45,914 : INFO : PROGRESS: at sentence #50000, processed 1119720 words, keeping 28041 word types
2017-05-12 18:06:46,006 : INFO : PROGRESS: at sentence #60000, processed 1343228 words, keeping 30340 word types
2017-05-12 18:06:46,073 : INFO : PROGRESS: at sentence #70000, processed 1566617 words, keeping 32273 word types
2017-05-12 18:06:46,140 : INFO : PROGRESS: at sentence #80000, processed 1790499 words, keeping 34241 word types
2017-05-12 18:06:46,209 : INFO : PROGRESS: at sentence #90000, processed 2013749 words, keeping 35926 word types
2017-05-12 18:06:46,273 : INFO : PROGRESS: at sentence #100000, processed 2236663 words, keeping 37438 word types
2017-05-12 18:06:46,333 : INFO : PROGRESS: at sentence #110000, processed 2460182 words, keeping 

In [9]:
for i in dimensions:
    w2w_model_strict = word2vec.Word2Vec.load('../../Results/Drosophila/models/dims/dros_strict_dim_parameter_'+str(i)+'_model')
    w2w_model_gen = word2vec.Word2Vec.load('../../Results/Drosophila/models/dims/dros_gen_dim_parameter_'+str(i)+'_model')
    w2w_model_be = word2vec.Word2Vec.load('../../Results/Drosophila/models/dims/dros_be_dim_parameter_'+str(i)+'_model')
    
    for seed in random_seeds:
        data_name = '../../Results/Drosophila/train_val/dros_tr_val_split_' + str(seed)
        train_data = pickle.load(open(data_name + '_train_data.pkl', 'rb'))
        train_labels = pickle.load(open(data_name + '_train_labels.pkl', 'rb'))
        validation_data = pickle.load(open(data_name + '_test_data.pkl', 'rb'))
        validation_labels = pickle.load(open(data_name + '_test_labels.pkl', 'rb'))
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_strict, feature_count=i)
        
        strict_list_SR_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                     train_labels, validation_labels]
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_gen, feature_count=i)
        
        strict_list_GEN_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                      train_labels, validation_labels]
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_be, feature_count=i)
        
        strict_list_BE_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                     train_labels, validation_labels]
        
        pickle.dump(strict_list_SR_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/dims/dros_strict_list_SR_dims_param_'+str(i)+'_'+str(seed)+'_results_list.pkl', 'wb'))
        pickle.dump(strict_list_GEN_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/dims/dros_strict_list_GEN_dims_param_'+str(i)+'_'+str(seed)+'_results_list.pkl', 'wb'))
        pickle.dump(strict_list_BE_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/dims/dros_strict_list_BE_dims_param_'+str(i)+'_'+str(seed)+'_results_list.pkl', 'wb'))
        
        strict_final_list = [strict_list_SR_dims_param, 
                             strict_list_GEN_dims_param, 
                             strict_list_BE_dims_param]
        print ('\nPredicting\n')
        errors = []
        fpr = []
        tpr = []

        for entry in strict_final_list:
            error_w2v_norm, probs_w2v_norm = pred.XGB_classifier(entry[0], entry[1],
                                                                 entry[2], entry[3])
            fpr_w2v_norm, tpr_w2v_norm, _ = roc_curve(entry[3], probs_w2v_norm)
            error_w2v_fs, probs_w2v_fs = pred.XGB_classifier(entry[0], entry[1],
                                                             entry[2], entry[3],
                                                             feature_selection=True)
            fpr_w2v_fs, tpr_w2v_fs, _ = roc_curve(entry[3], probs_w2v_fs)

            errors.append([error_w2v_norm, error_w2v_fs])
            fpr.append([fpr_w2v_norm, fpr_w2v_fs])
            tpr.append([tpr_w2v_norm, tpr_w2v_fs])
            
        pickle.dump(errors, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/dims/dros_dim_param'+str(i)+'_errors_pickle_'+str(seed)+'.pkl', 'wb'))
        pickle.dump(fpr, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/dims/dros_dim_param'+str(i)+'_fpr_pickle_'+str(seed)+'.pkl', 'wb'))
        pickle.dump(tpr, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/dims/dros_dim_param'+str(i)+'_tpr_pickle_'+str(seed)+'.pkl', 'wb'))

2017-05-12 18:22:56,642 : INFO : loading Word2Vec object from ../../Results/Drosophila/models/dims/dros_strict_dim_parameter_100_model
2017-05-12 18:22:56,958 : INFO : loading wv recursively from ../../Results/Drosophila/models/dims/dros_strict_dim_parameter_100_model.wv.* with mmap=None
2017-05-12 18:22:56,959 : INFO : setting ignored attribute syn0norm to None
2017-05-12 18:22:56,960 : INFO : setting ignored attribute cum_table to None
2017-05-12 18:22:56,961 : INFO : loaded ../../Results/Drosophila/models/dims/dros_strict_dim_parameter_100_model
2017-05-12 18:22:56,975 : INFO : loading Word2Vec object from ../../Results/Drosophila/models/dims/dros_gen_dim_parameter_100_model
2017-05-12 18:22:57,049 : INFO : loading wv recursively from ../../Results/Drosophila/models/dims/dros_gen_dim_parameter_100_model.wv.* with mmap=None
2017-05-12 18:22:57,054 : INFO : setting ignored attribute syn0norm to None
2017-05-12 18:22:57,055 : INFO : setting ignored attribute cum_table to None
2017-05-1


Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)


...


Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


In [11]:
def mult_open(direct, pattern):
    pickle_list = []
    file_list = []
    for f in os.listdir(direct):
        if re.search(pattern, f):
            f = f.split('_')
            f[-1] = f[-1][:-4]
            file_list.append(f)
    file_list.sort(key = lambda x: int(x[-1]))
    for file in file_list:
        file = '_'.join(file)
        file = file + '.pkl'
        pkl = pickle.load(open(os.path.join(direct, file), 'rb'))
        pickle_list.append(pkl)
    return pickle_list

In [12]:
best_parameter = []
for seed in random_seeds:
    mean_err_dros_strict = []
    mean_auc_dros_strict = []
    mean_err_dros_gen = []
    mean_auc_dros_gen = []
    mean_err_dros_be = []
    mean_auc_dros_be = []
    for i in dimensions:
        drct = '../../Results/Drosophila/error_fpr_tpr/w2v_param_search/dims/'
        errors_dros = mult_open(drct, str(i)+'_errors_pickle_'+str(seed))
        fpr_dros = mult_open(drct, str(i)+'_fpr_pickle_'+str(seed))
        tpr_dros = mult_open(drct, str(i)+'_tpr_pickle_'+str(seed))
        for e, f, t in zip(errors_dros, fpr_dros, tpr_dros):
            input_list = [[e, f, t]]
            name_list = ['drosophila']
            for idx in range(3):
                for item, name in zip(input_list, name_list):
                    for fpr_item, tpr_item, error_item in zip(item[1][idx], item[2][idx], item[0][idx]):
                        roc_auc = auc(fpr_item, tpr_item)
                        auc_val = '%.3f' % roc_auc
                        error = '%.3f' % error_item
                        if idx == 0:
                            mean_err_dros_strict.append(error_item)
                            mean_auc_dros_strict.append(roc_auc)
                        elif idx == 1:
                            mean_err_dros_gen.append(error_item)
                            mean_auc_dros_gen.append(roc_auc)
                        elif idx == 2:
                            mean_err_dros_be.append(error_item)
                            mean_auc_dros_be.append(roc_auc)

    mean_err_dros_org_strict = mean_err_dros_strict[0::2]
    mean_err_dros_fs_strict = mean_err_dros_strict[1::2]
    mean_auc_dros_org_strict = mean_auc_dros_strict[0::2]
    mean_auc_dros_fs_strict = mean_auc_dros_strict[1::2]

    mean_err_dros_org_gen = mean_err_dros_gen[0::2]
    mean_err_dros_fs_gen = mean_err_dros_gen[1::2]
    mean_auc_dros_org_gen = mean_auc_dros_gen[0::2]
    mean_auc_dros_fs_gen = mean_auc_dros_gen[1::2]

    mean_err_dros_org_be = mean_err_dros_be[0::2]
    mean_err_dros_fs_be = mean_err_dros_be[1::2]
    mean_auc_dros_org_be = mean_auc_dros_be[0::2]
    mean_auc_dros_fs_be = mean_auc_dros_be[1::2]
    
    org_error = max([(mean_err_dros_org_strict.index(min(mean_err_dros_org_strict))+1)*100, 
                     (mean_err_dros_org_gen.index(min(mean_err_dros_org_gen))+1)*100, 
                     (mean_err_dros_org_be.index(min(mean_err_dros_org_be))+1)*100])
    
    fs_error = max([(mean_err_dros_fs_strict.index(min(mean_err_dros_fs_strict))+1)*100, 
                     (mean_err_dros_fs_gen.index(min(mean_err_dros_fs_gen))+1)*100, 
                     (mean_err_dros_fs_be.index(min(mean_err_dros_fs_be))+1)*100])
    
    org_auc = max([(mean_auc_dros_org_strict.index(max(mean_auc_dros_org_strict))+1)*100, 
                     (mean_auc_dros_org_gen.index(max(mean_auc_dros_org_gen))+1)*100, 
                     (mean_auc_dros_org_be.index(max(mean_auc_dros_org_be))+1)*100])
    
    fs_auc = max([(mean_auc_dros_fs_strict.index(max(mean_auc_dros_fs_strict))+1)*100, 
                     (mean_auc_dros_fs_gen.index(max(mean_auc_dros_fs_gen))+1)*100, 
                     (mean_auc_dros_fs_be.index(max(mean_auc_dros_fs_be))+1)*100])
    
    best_parameter.append((seed, [org_error, fs_error, org_auc, fs_auc]))
    

In [13]:
def most_common(lst):
    return max(set(lst), key=lst.count)

best_overall = []
for i in best_parameter:
    best_overall.append(most_common(i[1]))
    print(i)
best_dims = most_common(best_overall)
print(best_dims)

(144, [400, 200, 500, 400])
(235, [300, 300, 600, 600])
(905, [400, 100, 500, 500])
(2895, [300, 300, 200, 600])
(3462, [400, 200, 600, 600])
(4225, [600, 300, 600, 600])
(5056, [600, 200, 600, 500])
(5192, [600, 600, 600, 600])
(7751, [100, 500, 300, 200])
(7813, [300, 600, 300, 500])
600


In [35]:
min_word_count = [1, 2, 3, 4, 5, 6]
for i in min_word_count:
    w2v_parameters = [600, i, 4, 6, 0.001]
    strict = pred.make_w2v_model(dros_strict_real, 'Drosophila/models/word_count/dros_strict_word_count_parameter_' + str(i), w2v_parameters)
    gen = pred.make_w2v_model(dros_gen_real, 'Drosophila/models/word_count/dros_gen_word_count_parameter_' + str(i), w2v_parameters)
    be = pred.make_w2v_model(dros_be_real, 'Drosophila/models/word_count/dros_be_word_count_parameter_' + str(i), w2v_parameters)

2017-05-12 19:39:04,959 : INFO : collecting all words and their counts
2017-05-12 19:39:04,961 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-12 19:39:05,002 : INFO : collected 12999 word types from a corpus of 188627 raw words and 7408 sentences
2017-05-12 19:39:05,004 : INFO : Loading a fresh vocabulary
2017-05-12 19:39:05,041 : INFO : min_count=1 retains 12999 unique words (100% of original 12999, drops 0)
2017-05-12 19:39:05,042 : INFO : min_count=1 leaves 188627 word corpus (100% of original 188627, drops 0)


Parsing datasets sentences
Training Word2Vec Model


2017-05-12 19:39:05,078 : INFO : deleting the raw counts dictionary of 12999 items
2017-05-12 19:39:05,080 : INFO : sample=0.001 downsamples 32 most-common words
2017-05-12 19:39:05,080 : INFO : downsampling leaves estimated 141308 word corpus (74.9% of prior 188627)
2017-05-12 19:39:05,081 : INFO : estimated required memory for 12999 words and 600 dimensions: 68894700 bytes
2017-05-12 19:39:05,138 : INFO : resetting layer weights
2017-05-12 19:39:05,380 : INFO : training model with 4 workers on 12999 vocabulary and 600 features, using sg=0 hs=0 sample=0.001 negative=5 window=6
2017-05-12 19:39:05,381 : INFO : expecting 7408 sentences, matching count from corpus used for vocabulary survey
2017-05-12 19:39:06,398 : INFO : PROGRESS: at 28.58% examples, 199687 words/s, in_qsize 8, out_qsize 0
2017-05-12 19:39:07,425 : INFO : PROGRESS: at 57.22% examples, 198239 words/s, in_qsize 8, out_qsize 0
2017-05-12 19:39:08,450 : INFO : PROGRESS: at 87.88% examples, 202689 words/s, in_qsize 7, out_q

Parsing datasets sentences


2017-05-12 19:39:10,411 : INFO : collecting all words and their counts
2017-05-12 19:39:10,413 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-12 19:39:10,482 : INFO : PROGRESS: at sentence #10000, processed 240061 words, keeping 13294 word types
2017-05-12 19:39:10,547 : INFO : PROGRESS: at sentence #20000, processed 479451 words, keeping 17608 word types


...

2017-05-12 19:48:17,102 : INFO : PROGRESS: at sentence #30000, processed 719251 words, keeping 20837 word types
2017-05-12 19:48:17,159 : INFO : collected 22824 word types from a corpus of 904687 raw words and 37697 sentences
2017-05-12 19:48:17,160 : INFO : Loading a fresh vocabulary
2017-05-12 19:48:17,190 : INFO : min_count=6 retains 7287 unique words (31% of original 22824, drops 15537)
2017-05-12 19:48:17,191 : INFO : min_count=6 leaves 875943 word corpus (96% of original 904687, drops 28744)
2017-05-12 19:48:17,212 : INFO : deleting the raw counts dictionary of 22824 items
2017-05-12 19:48:17,214 : INFO : sample=0.001 downsamples 40 most-common words
2017-05-12 19:48:17,216 : INFO : downsampling leaves estimated 640540 word corpus (73.1% of prior 875943)
2017-05-12 19:48:17,217 : INFO : estimated required memory for 7287 words and 600 dimensions: 38621100 bytes
2017-05-12 19:48:17,251 : INFO : resetting layer weights
2017-05-12 19:48:17,385 : INFO : training model with 4 workers 

Parsing datasets sentences


2017-05-12 19:48:34,002 : INFO : collecting all words and their counts
2017-05-12 19:48:34,004 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-12 19:48:34,068 : INFO : PROGRESS: at sentence #10000, processed 223978 words, keeping 13703 word types
2017-05-12 19:48:34,128 : INFO : PROGRESS: at sentence #20000, processed 448086 words, keeping 18835 word types
2017-05-12 19:48:34,197 : INFO : PROGRESS: at sentence #30000, processed 672056 words, keeping 22546 word types


Training Word2Vec Model


2017-05-12 19:48:34,264 : INFO : PROGRESS: at sentence #40000, processed 896507 words, keeping 25560 word types
2017-05-12 19:48:34,322 : INFO : PROGRESS: at sentence #50000, processed 1119720 words, keeping 28041 word types
2017-05-12 19:48:34,388 : INFO : PROGRESS: at sentence #60000, processed 1343228 words, keeping 30340 word types
2017-05-12 19:48:34,469 : INFO : PROGRESS: at sentence #70000, processed 1566617 words, keeping 32273 word types
2017-05-12 19:48:34,536 : INFO : PROGRESS: at sentence #80000, processed 1790499 words, keeping 34241 word types
2017-05-12 19:48:34,601 : INFO : PROGRESS: at sentence #90000, processed 2013749 words, keeping 35926 word types
2017-05-12 19:48:34,669 : INFO : PROGRESS: at sentence #100000, processed 2236663 words, keeping 37438 word types
2017-05-12 19:48:34,735 : INFO : PROGRESS: at sentence #110000, processed 2460182 words, keeping 38847 word types
2017-05-12 19:48:34,797 : INFO : PROGRESS: at sentence #120000, processed 2683421 words, keepin

In [38]:
for i in min_word_count:
    w2w_model_strict = word2vec.Word2Vec.load('../../Results/Drosophila/models/word_count/dros_strict_word_count_parameter_'+str(i)+'_model')
    w2w_model_gen = word2vec.Word2Vec.load('../../Results/Drosophila/models/word_count/dros_gen_word_count_parameter_'+str(i)+'_model')
    w2w_model_be = word2vec.Word2Vec.load('../../Results/Drosophila/models/word_count/dros_be_word_count_parameter_'+str(i)+'_model')
    
    for seed in random_seeds:
        data_name = '../../Results/Drosophila/train_val/dros_tr_val_split_' + str(seed)
        train_data = pickle.load(open(data_name + '_train_data.pkl', 'rb'))
        train_labels = pickle.load(open(data_name + '_train_labels.pkl', 'rb'))
        validation_data = pickle.load(open(data_name + '_test_data.pkl', 'rb'))
        validation_labels = pickle.load(open(data_name + '_test_labels.pkl', 'rb'))
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_strict, feature_count=600)
        
        strict_list_SR_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                     train_labels, validation_labels]
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_gen, feature_count=600)
        
        strict_list_GEN_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                      train_labels, validation_labels]
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_be, feature_count=600)
        
        strict_list_BE_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                     train_labels, validation_labels]
        
        pickle.dump(strict_list_SR_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/word_count/dros_strict_list_SR_word_count_param_'+str(i)+'_'+str(seed)+'_results_list.pkl', 'wb'))
        pickle.dump(strict_list_GEN_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/word_count/dros_strict_list_GEN_word_count_param_'+str(i)+'_'+str(seed)+'_results_list.pkl', 'wb'))
        pickle.dump(strict_list_BE_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/word_count/dros_strict_list_BE_word_count_param_'+str(i)+'_'+str(seed)+'_results_list.pkl', 'wb'))
        
        strict_final_list = [strict_list_SR_dims_param, 
                             strict_list_GEN_dims_param, 
                             strict_list_BE_dims_param]
        print ('\nPredicting\n')
        errors = []
        fpr = []
        tpr = []

        for entry in strict_final_list:
            error_w2v_norm, probs_w2v_norm = pred.XGB_classifier(entry[0], entry[1],
                                                                 entry[2], entry[3])
            fpr_w2v_norm, tpr_w2v_norm, _ = roc_curve(entry[3], probs_w2v_norm)
            error_w2v_fs, probs_w2v_fs = pred.XGB_classifier(entry[0], entry[1],
                                                             entry[2], entry[3],
                                                             feature_selection=True)
            fpr_w2v_fs, tpr_w2v_fs, _ = roc_curve(entry[3], probs_w2v_fs)

            errors.append([error_w2v_norm, error_w2v_fs])
            fpr.append([fpr_w2v_norm, fpr_w2v_fs])
            tpr.append([tpr_w2v_norm, tpr_w2v_fs])
            
        pickle.dump(errors, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/word_count/dros_word_count_param'+str(i)+'_errors_pickle_'+str(seed)+'.pkl', 'wb'))
        pickle.dump(fpr, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/word_count/dros_word_count_param'+str(i)+'_fpr_pickle_'+str(seed)+'.pkl', 'wb'))
        pickle.dump(tpr, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/word_count/dros_word_count_param'+str(i)+'_tpr_pickle_'+str(seed)+'.pkl', 'wb'))

2017-05-12 19:52:05,001 : INFO : loading Word2Vec object from ../../Results/Drosophila/models/word_count/dros_strict_word_count_parameter_1_model
2017-05-12 19:52:05,542 : INFO : loading wv recursively from ../../Results/Drosophila/models/word_count/dros_strict_word_count_parameter_1_model.wv.* with mmap=None
2017-05-12 19:52:05,544 : INFO : setting ignored attribute syn0norm to None
2017-05-12 19:52:05,546 : INFO : setting ignored attribute cum_table to None
2017-05-12 19:52:05,546 : INFO : loaded ../../Results/Drosophila/models/word_count/dros_strict_word_count_parameter_1_model
2017-05-12 19:52:05,582 : INFO : loading Word2Vec object from ../../Results/Drosophila/models/word_count/dros_gen_word_count_parameter_1_model
2017-05-12 19:52:05,648 : INFO : loading wv recursively from ../../Results/Drosophila/models/word_count/dros_gen_word_count_parameter_1_model.wv.* with mmap=None
2017-05-12 19:52:05,651 : INFO : loading syn0 from ../../Results/Drosophila/models/word_count/dros_gen_word


Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 M

../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)


...


Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


In [14]:
best_parameter = []
for seed in random_seeds:
    mean_err_dros_strict = []
    mean_auc_dros_strict = []
    mean_err_dros_gen = []
    mean_auc_dros_gen = []
    mean_err_dros_be = []
    mean_auc_dros_be = []
    for i in min_word_count:
        drct = '../../Results/Drosophila/error_fpr_tpr/w2v_param_search/word_count/'
        errors_dros = mult_open(drct, str(i)+'_errors_pickle_'+str(seed))
        fpr_dros = mult_open(drct, str(i)+'_fpr_pickle_'+str(seed))
        tpr_dros = mult_open(drct, str(i)+'_tpr_pickle_'+str(seed))
        for e, f, t in zip(errors_dros, fpr_dros, tpr_dros):
            input_list = [[e, f, t]]
            name_list = ['drosophila']
            for idx in range(3):
                for item, name in zip(input_list, name_list):
                    for fpr_item, tpr_item, error_item in zip(item[1][idx], item[2][idx], item[0][idx]):
                        roc_auc = auc(fpr_item, tpr_item)
                        auc_val = '%.3f' % roc_auc
                        error = '%.3f' % error_item
                        if idx == 0:
                            mean_err_dros_strict.append(error_item)
                            mean_auc_dros_strict.append(roc_auc)
                        elif idx == 1:
                            mean_err_dros_gen.append(error_item)
                            mean_auc_dros_gen.append(roc_auc)
                        elif idx == 2:
                            mean_err_dros_be.append(error_item)
                            mean_auc_dros_be.append(roc_auc)

    mean_err_dros_org_strict = mean_err_dros_strict[0::2]
    mean_err_dros_fs_strict = mean_err_dros_strict[1::2]
    mean_auc_dros_org_strict = mean_auc_dros_strict[0::2]
    mean_auc_dros_fs_strict = mean_auc_dros_strict[1::2]

    mean_err_dros_org_gen = mean_err_dros_gen[0::2]
    mean_err_dros_fs_gen = mean_err_dros_gen[1::2]
    mean_auc_dros_org_gen = mean_auc_dros_gen[0::2]
    mean_auc_dros_fs_gen = mean_auc_dros_gen[1::2]

    mean_err_dros_org_be = mean_err_dros_be[0::2]
    mean_err_dros_fs_be = mean_err_dros_be[1::2]
    mean_auc_dros_org_be = mean_auc_dros_be[0::2]
    mean_auc_dros_fs_be = mean_auc_dros_be[1::2]
    
    org_error = max([(mean_err_dros_org_strict.index(min(mean_err_dros_org_strict))+1), 
                     (mean_err_dros_org_gen.index(min(mean_err_dros_org_gen))+1), 
                     (mean_err_dros_org_be.index(min(mean_err_dros_org_be))+1)])
    
    fs_error = max([(mean_err_dros_fs_strict.index(min(mean_err_dros_fs_strict))+1), 
                     (mean_err_dros_fs_gen.index(min(mean_err_dros_fs_gen))+1), 
                     (mean_err_dros_fs_be.index(min(mean_err_dros_fs_be))+1)])
    
    org_auc = max([(mean_auc_dros_org_strict.index(max(mean_auc_dros_org_strict))+1), 
                     (mean_auc_dros_org_gen.index(max(mean_auc_dros_org_gen))+1), 
                     (mean_auc_dros_org_be.index(max(mean_auc_dros_org_be))+1)])
    
    fs_auc = max([(mean_auc_dros_fs_strict.index(max(mean_auc_dros_fs_strict))+1), 
                     (mean_auc_dros_fs_gen.index(max(mean_auc_dros_fs_gen))+1), 
                     (mean_auc_dros_fs_be.index(max(mean_auc_dros_fs_be))+1)])
    
    best_parameter.append((seed, [org_error, fs_error, org_auc, fs_auc]))

In [15]:
best_overall = []
for i in best_parameter:
    best_overall.append(most_common(i[1]))
    print(i)
best_word_count = most_common(best_overall)
print(best_word_count)

(144, [6, 2, 5, 6])
(235, [2, 4, 6, 2])
(905, [6, 3, 5, 5])
(2895, [1, 4, 4, 3])
(3462, [1, 6, 6, 5])
(4225, [4, 5, 6, 6])
(5056, [4, 4, 4, 4])
(5192, [5, 2, 6, 5])
(7751, [6, 6, 6, 6])
(7813, [6, 4, 4, 2])
6


In [60]:
context_window = [2, 3, 4, 5, 6, 7]
for i in context_window:
    w2v_parameters = [best_dims, best_word_count, 4, i, 0.001]
    strict = pred.make_w2v_model(dros_strict_real, 'Drosophila/models/context_window/dros_strict_context_window_parameter_' + str(i), w2v_parameters)
    gen = pred.make_w2v_model(dros_gen_real, 'Drosophila/models/context_window/dros_gen_context_window_parameter_' + str(i), w2v_parameters)
    be = pred.make_w2v_model(dros_be_real, 'Drosophila/models/context_window/dros_be_context_window_parameter_' + str(i), w2v_parameters)

2017-05-12 20:44:15,195 : INFO : collecting all words and their counts
2017-05-12 20:44:15,195 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-12 20:44:15,232 : INFO : collected 12999 word types from a corpus of 188627 raw words and 7408 sentences
2017-05-12 20:44:15,233 : INFO : Loading a fresh vocabulary
2017-05-12 20:44:15,249 : INFO : min_count=6 retains 3074 unique words (23% of original 12999, drops 9925)
2017-05-12 20:44:15,249 : INFO : min_count=6 leaves 170913 word corpus (90% of original 188627, drops 17714)
2017-05-12 20:44:15,256 : INFO : deleting the raw counts dictionary of 12999 items
2017-05-12 20:44:15,257 : INFO : sample=0.001 downsamples 38 most-common words
2017-05-12 20:44:15,258 : INFO : downsampling leaves estimated 122076 word corpus (71.4% of prior 170913)
2017-05-12 20:44:15,258 : INFO : estimated required memory for 3074 words and 600 dimensions: 16292200 bytes
2017-05-12 20:44:15,265 : INFO : resetting layer weights


Parsing datasets sentences
Training Word2Vec Model


2017-05-12 20:44:15,316 : INFO : training model with 4 workers on 3074 vocabulary and 600 features, using sg=0 hs=0 sample=0.001 negative=5 window=2
2017-05-12 20:44:15,317 : INFO : expecting 7408 sentences, matching count from corpus used for vocabulary survey
2017-05-12 20:44:16,184 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-05-12 20:44:16,192 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-05-12 20:44:16,194 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-05-12 20:44:16,199 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-05-12 20:44:16,200 : INFO : training on 943135 raw words (610641 effective words) took 0.9s, 695629 effective words/s
2017-05-12 20:44:16,201 : INFO : precomputing L2-norms of word weight vectors
2017-05-12 20:44:16,222 : INFO : saving Word2Vec object under ../../Results/Drosophila/models/context_window/dros_strict_context_window_parameter_2_model, separately None
20

Parsing datasets sentences


2017-05-12 20:44:16,768 : INFO : collecting all words and their counts
2017-05-12 20:44:16,769 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-12 20:44:16,819 : INFO : PROGRESS: at sentence #10000, processed 240061 words, keeping 13294 word types
2017-05-12 20:44:16,881 : INFO : PROGRESS: at sentence #20000, processed 479451 words, keeping 17608 word types
2017-05-12 20:44:16,942 : INFO : PROGRESS: at sentence #30000, processed 719251 words, keeping 20837 word types


...

2017-05-12 20:49:10,278 : INFO : min_count=6 retains 7287 unique words (31% of original 22824, drops 15537)
2017-05-12 20:49:10,278 : INFO : min_count=6 leaves 875943 word corpus (96% of original 904687, drops 28744)
2017-05-12 20:49:10,299 : INFO : deleting the raw counts dictionary of 22824 items
2017-05-12 20:49:10,300 : INFO : sample=0.001 downsamples 40 most-common words
2017-05-12 20:49:10,301 : INFO : downsampling leaves estimated 640540 word corpus (73.1% of prior 875943)
2017-05-12 20:49:10,302 : INFO : estimated required memory for 7287 words and 600 dimensions: 38621100 bytes
2017-05-12 20:49:10,333 : INFO : resetting layer weights
2017-05-12 20:49:10,463 : INFO : training model with 4 workers on 7287 vocabulary and 600 features, using sg=0 hs=0 sample=0.001 negative=5 window=7
2017-05-12 20:49:10,464 : INFO : expecting 37697 sentences, matching count from corpus used for vocabulary survey
2017-05-12 20:49:11,471 : INFO : PROGRESS: at 11.93% examples, 381044 words/s, in_qsiz

Parsing datasets sentences


2017-05-12 20:49:23,290 : INFO : collecting all words and their counts
2017-05-12 20:49:23,293 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-12 20:49:23,351 : INFO : PROGRESS: at sentence #10000, processed 223978 words, keeping 13703 word types
2017-05-12 20:49:23,412 : INFO : PROGRESS: at sentence #20000, processed 448086 words, keeping 18835 word types
2017-05-12 20:49:23,473 : INFO : PROGRESS: at sentence #30000, processed 672056 words, keeping 22546 word types


Training Word2Vec Model


2017-05-12 20:49:23,534 : INFO : PROGRESS: at sentence #40000, processed 896507 words, keeping 25560 word types
2017-05-12 20:49:23,596 : INFO : PROGRESS: at sentence #50000, processed 1119720 words, keeping 28041 word types
2017-05-12 20:49:23,656 : INFO : PROGRESS: at sentence #60000, processed 1343228 words, keeping 30340 word types
2017-05-12 20:49:23,714 : INFO : PROGRESS: at sentence #70000, processed 1566617 words, keeping 32273 word types
2017-05-12 20:49:23,775 : INFO : PROGRESS: at sentence #80000, processed 1790499 words, keeping 34241 word types
2017-05-12 20:49:23,837 : INFO : PROGRESS: at sentence #90000, processed 2013749 words, keeping 35926 word types
2017-05-12 20:49:23,884 : INFO : PROGRESS: at sentence #100000, processed 2236663 words, keeping 37438 word types
2017-05-12 20:49:23,937 : INFO : PROGRESS: at sentence #110000, processed 2460182 words, keeping 38847 word types
2017-05-12 20:49:23,987 : INFO : PROGRESS: at sentence #120000, processed 2683421 words, keepin

In [61]:
for i in context_window:
    w2w_model_strict = word2vec.Word2Vec.load('../../Results/Drosophila/models/context_window/dros_strict_context_window_parameter_'+str(i)+'_model')
    w2w_model_gen = word2vec.Word2Vec.load('../../Results/Drosophila/models/context_window/dros_gen_context_window_parameter_'+str(i)+'_model')
    w2w_model_be = word2vec.Word2Vec.load('../../Results/Drosophila/models/context_window/dros_be_context_window_parameter_'+str(i)+'_model')
    
    for seed in random_seeds:
        data_name = '../../Results/Drosophila/train_val/dros_tr_val_split_' + str(seed)
        train_data = pickle.load(open(data_name + '_train_data.pkl', 'rb'))
        train_labels = pickle.load(open(data_name + '_train_labels.pkl', 'rb'))
        validation_data = pickle.load(open(data_name + '_test_data.pkl', 'rb'))
        validation_labels = pickle.load(open(data_name + '_test_labels.pkl', 'rb'))
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_strict, feature_count=600)
        
        strict_list_SR_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                     train_labels, validation_labels]
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_gen, feature_count=600)
        
        strict_list_GEN_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                      train_labels, validation_labels]
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_be, feature_count=600)
        
        strict_list_BE_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                     train_labels, validation_labels]
        
        pickle.dump(strict_list_SR_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/context_window/dros_strict_list_SR_context_window_param_'+str(i)+'_'+str(seed)+'_results_list.pkl', 'wb'))
        pickle.dump(strict_list_GEN_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/context_window/dros_strict_list_GEN_context_window_param_'+str(i)+'_'+str(seed)+'_results_list.pkl', 'wb'))
        pickle.dump(strict_list_BE_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/context_window/dros_strict_list_BE_context_window_param_'+str(i)+'_'+str(seed)+'_results_list.pkl', 'wb'))
        
        strict_final_list = [strict_list_SR_dims_param, 
                             strict_list_GEN_dims_param, 
                             strict_list_BE_dims_param]
        print ('\nPredicting\n')
        errors = []
        fpr = []
        tpr = []

        for entry in strict_final_list:
            error_w2v_norm, probs_w2v_norm = pred.XGB_classifier(entry[0], entry[1],
                                                                 entry[2], entry[3])
            fpr_w2v_norm, tpr_w2v_norm, _ = roc_curve(entry[3], probs_w2v_norm)
            error_w2v_fs, probs_w2v_fs = pred.XGB_classifier(entry[0], entry[1],
                                                             entry[2], entry[3],
                                                             feature_selection=True)
            fpr_w2v_fs, tpr_w2v_fs, _ = roc_curve(entry[3], probs_w2v_fs)

            errors.append([error_w2v_norm, error_w2v_fs])
            fpr.append([fpr_w2v_norm, fpr_w2v_fs])
            tpr.append([tpr_w2v_norm, tpr_w2v_fs])
            
        pickle.dump(errors, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/context_window/dros_context_window_param'+str(i)+'_errors_pickle_'+str(seed)+'.pkl', 'wb'))
        pickle.dump(fpr, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/context_window/dros_context_window_param'+str(i)+'_fpr_pickle_'+str(seed)+'.pkl', 'wb'))
        pickle.dump(tpr, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/context_window/dros_context_window_param'+str(i)+'_tpr_pickle_'+str(seed)+'.pkl', 'wb'))

2017-05-12 20:50:18,639 : INFO : loading Word2Vec object from ../../Results/Drosophila/models/context_window/dros_strict_context_window_parameter_2_model
2017-05-12 20:50:18,737 : INFO : loading wv recursively from ../../Results/Drosophila/models/context_window/dros_strict_context_window_parameter_2_model.wv.* with mmap=None
2017-05-12 20:50:18,738 : INFO : setting ignored attribute syn0norm to None
2017-05-12 20:50:18,739 : INFO : setting ignored attribute cum_table to None
2017-05-12 20:50:18,739 : INFO : loaded ../../Results/Drosophila/models/context_window/dros_strict_context_window_parameter_2_model
2017-05-12 20:50:18,748 : INFO : loading Word2Vec object from ../../Results/Drosophila/models/context_window/dros_gen_context_window_parameter_2_model
2017-05-12 20:50:18,978 : INFO : loading wv recursively from ../../Results/Drosophila/models/context_window/dros_gen_context_window_parameter_2_model.wv.* with mmap=None
2017-05-12 20:50:18,979 : INFO : setting ignored attribute syn0norm


Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)


...


Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


In [16]:
best_parameter = []
for seed in random_seeds:
    mean_err_dros_strict = []
    mean_auc_dros_strict = []
    mean_err_dros_gen = []
    mean_auc_dros_gen = []
    mean_err_dros_be = []
    mean_auc_dros_be = []
    for i in context_window:
        drct = '../../Results/Drosophila/error_fpr_tpr/w2v_param_search/context_window/'
        errors_dros = mult_open(drct, str(i)+'_errors_pickle_'+str(seed))
        fpr_dros = mult_open(drct, str(i)+'_fpr_pickle_'+str(seed))
        tpr_dros = mult_open(drct, str(i)+'_tpr_pickle_'+str(seed))
        for e, f, t in zip(errors_dros, fpr_dros, tpr_dros):
            input_list = [[e, f, t]]
            name_list = ['drosophila']
            for idx in range(3):
                for item, name in zip(input_list, name_list):
                    for fpr_item, tpr_item, error_item in zip(item[1][idx], item[2][idx], item[0][idx]):
                        roc_auc = auc(fpr_item, tpr_item)
                        auc_val = '%.3f' % roc_auc
                        error = '%.3f' % error_item
                        if idx == 0:
                            mean_err_dros_strict.append(error_item)
                            mean_auc_dros_strict.append(roc_auc)
                        elif idx == 1:
                            mean_err_dros_gen.append(error_item)
                            mean_auc_dros_gen.append(roc_auc)
                        elif idx == 2:
                            mean_err_dros_be.append(error_item)
                            mean_auc_dros_be.append(roc_auc)

    mean_err_dros_org_strict = mean_err_dros_strict[0::2]
    mean_err_dros_fs_strict = mean_err_dros_strict[1::2]
    mean_auc_dros_org_strict = mean_auc_dros_strict[0::2]
    mean_auc_dros_fs_strict = mean_auc_dros_strict[1::2]

    mean_err_dros_org_gen = mean_err_dros_gen[0::2]
    mean_err_dros_fs_gen = mean_err_dros_gen[1::2]
    mean_auc_dros_org_gen = mean_auc_dros_gen[0::2]
    mean_auc_dros_fs_gen = mean_auc_dros_gen[1::2]

    mean_err_dros_org_be = mean_err_dros_be[0::2]
    mean_err_dros_fs_be = mean_err_dros_be[1::2]
    mean_auc_dros_org_be = mean_auc_dros_be[0::2]
    mean_auc_dros_fs_be = mean_auc_dros_be[1::2]
    
    org_error = max([(mean_err_dros_org_strict.index(min(mean_err_dros_org_strict))+1), 
                     (mean_err_dros_org_gen.index(min(mean_err_dros_org_gen))+1), 
                     (mean_err_dros_org_be.index(min(mean_err_dros_org_be))+1)])
    
    fs_error = max([(mean_err_dros_fs_strict.index(min(mean_err_dros_fs_strict))+1), 
                     (mean_err_dros_fs_gen.index(min(mean_err_dros_fs_gen))+1), 
                     (mean_err_dros_fs_be.index(min(mean_err_dros_fs_be))+1)])
    
    org_auc = max([(mean_auc_dros_org_strict.index(max(mean_auc_dros_org_strict))+1), 
                     (mean_auc_dros_org_gen.index(max(mean_auc_dros_org_gen))+1), 
                     (mean_auc_dros_org_be.index(max(mean_auc_dros_org_be))+1)])
    
    fs_auc = max([(mean_auc_dros_fs_strict.index(max(mean_auc_dros_fs_strict))+1), 
                     (mean_auc_dros_fs_gen.index(max(mean_auc_dros_fs_gen))+1), 
                     (mean_auc_dros_fs_be.index(max(mean_auc_dros_fs_be))+1)])
    
    best_parameter.append((seed, [org_error, fs_error, org_auc, fs_auc]))

In [17]:
best_overall = []
for i in best_parameter:
    best_overall.append(most_common(i[1]))
    print(i)
best_context_window = most_common(best_overall)
print(best_context_window)

(144, [1, 6, 6, 3])
(235, [1, 5, 6, 6])
(905, [3, 4, 6, 5])
(2895, [4, 1, 5, 3])
(3462, [5, 5, 4, 4])
(4225, [6, 5, 5, 5])
(5056, [5, 6, 6, 5])
(5192, [5, 4, 2, 2])
(7751, [4, 6, 6, 6])
(7813, [4, 6, 3, 4])
6


In [32]:
downsample = [(1, 0.1), (2, 0.01), (3, 0.001), (4, 0.0001), (5, 0.00001), (6, 0.000001)]
for i in downsample:
    w2v_parameters = [best_dims, best_word_count, 4, best_context_window, i[1]]
    strict = pred.make_w2v_model(dros_strict_real, 'Drosophila/models/downsample/dros_strict_downsample_parameter_' + str(i[0]), w2v_parameters)
    gen = pred.make_w2v_model(dros_gen_real, 'Drosophila/models/downsample/dros_gen_downsample_parameter_' + str(i[0]), w2v_parameters)
    be = pred.make_w2v_model(dros_be_real, 'Drosophila/models/downsample/dros_be_downsample_parameter_' + str(i[0]), w2v_parameters)

2017-05-13 11:00:03,120 : INFO : collecting all words and their counts
2017-05-13 11:00:03,121 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-13 11:00:03,154 : INFO : collected 12999 word types from a corpus of 188627 raw words and 7408 sentences
2017-05-13 11:00:03,155 : INFO : Loading a fresh vocabulary
2017-05-13 11:00:03,169 : INFO : min_count=6 retains 3074 unique words (23% of original 12999, drops 9925)
2017-05-13 11:00:03,170 : INFO : min_count=6 leaves 170913 word corpus (90% of original 188627, drops 17714)
2017-05-13 11:00:03,177 : INFO : deleting the raw counts dictionary of 12999 items
2017-05-13 11:00:03,178 : INFO : sample=0.1 downsamples 0 most-common words
2017-05-13 11:00:03,178 : INFO : downsampling leaves estimated 170913 word corpus (100.0% of prior 170913)
2017-05-13 11:00:03,179 : INFO : estimated required memory for 3074 words and 600 dimensions: 16292200 bytes
2017-05-13 11:00:03,188 : INFO : resetting layer weights
2017-05-1

Parsing datasets sentences
Training Word2Vec Model


2017-05-13 11:00:03,236 : INFO : expecting 7408 sentences, matching count from corpus used for vocabulary survey
2017-05-13 11:00:04,263 : INFO : PROGRESS: at 75.09% examples, 628871 words/s, in_qsize 7, out_qsize 0
2017-05-13 11:00:04,561 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-05-13 11:00:04,568 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-05-13 11:00:04,578 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-05-13 11:00:04,585 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-05-13 11:00:04,586 : INFO : training on 943135 raw words (854565 effective words) took 1.3s, 635647 effective words/s
2017-05-13 11:00:04,587 : INFO : precomputing L2-norms of word weight vectors
2017-05-13 11:00:04,606 : INFO : saving Word2Vec object under ../../Results/Drosophila/models/downsample/dros_strict_downsample_parameter_1_model, separately None
2017-05-13 11:00:04,607 : INFO : not storing attribute s

Parsing datasets sentences


2017-05-13 11:00:05,181 : INFO : collecting all words and their counts
2017-05-13 11:00:05,182 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-13 11:00:05,238 : INFO : PROGRESS: at sentence #10000, processed 240504 words, keeping 13182 word types
2017-05-13 11:00:05,291 : INFO : PROGRESS: at sentence #20000, processed 479354 words, keeping 17779 word types
2017-05-13 11:00:05,337 : INFO : PROGRESS: at sentence #30000, processed 719840 words, keeping 20864 word types
2017-05-13 11:00:05,374 : INFO : collected 22824 word types from a corpus of 904687 raw words and 37697 sentences
2017-05-13 11:00:05,375 : INFO : Loading a fresh vocabulary


...

2017-05-13 11:04:02,673 : INFO : deleting the raw counts dictionary of 22824 items
2017-05-13 11:04:02,675 : INFO : sample=1e-06 downsamples 7287 most-common words
2017-05-13 11:04:02,676 : INFO : downsampling leaves estimated 52117 word corpus (5.9% of prior 875943)
2017-05-13 11:04:02,677 : INFO : estimated required memory for 7287 words and 600 dimensions: 38621100 bytes
2017-05-13 11:04:02,699 : INFO : resetting layer weights
2017-05-13 11:04:02,807 : INFO : training model with 4 workers on 7287 vocabulary and 600 features, using sg=0 hs=0 sample=1e-06 negative=5 window=6
2017-05-13 11:04:02,808 : INFO : expecting 37697 sentences, matching count from corpus used for vocabulary survey
2017-05-13 11:04:03,818 : INFO : PROGRESS: at 53.87% examples, 139719 words/s, in_qsize 8, out_qsize 0
2017-05-13 11:04:04,640 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-05-13 11:04:04,641 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-05-13 11:04:0

Parsing datasets sentences


2017-05-13 11:04:06,877 : INFO : collecting all words and their counts
2017-05-13 11:04:06,878 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-13 11:04:06,921 : INFO : PROGRESS: at sentence #10000, processed 225415 words, keeping 13684 word types
2017-05-13 11:04:06,962 : INFO : PROGRESS: at sentence #20000, processed 449496 words, keeping 18758 word types
2017-05-13 11:04:07,002 : INFO : PROGRESS: at sentence #30000, processed 672246 words, keeping 22544 word types
2017-05-13 11:04:07,043 : INFO : PROGRESS: at sentence #40000, processed 895570 words, keeping 25577 word types


Training Word2Vec Model


2017-05-13 11:04:07,084 : INFO : PROGRESS: at sentence #50000, processed 1119209 words, keeping 28211 word types
2017-05-13 11:04:07,126 : INFO : PROGRESS: at sentence #60000, processed 1341936 words, keeping 30318 word types
2017-05-13 11:04:07,167 : INFO : PROGRESS: at sentence #70000, processed 1563772 words, keeping 32389 word types
2017-05-13 11:04:07,208 : INFO : PROGRESS: at sentence #80000, processed 1787418 words, keeping 34236 word types
2017-05-13 11:04:07,253 : INFO : PROGRESS: at sentence #90000, processed 2009809 words, keeping 35914 word types
2017-05-13 11:04:07,294 : INFO : PROGRESS: at sentence #100000, processed 2233773 words, keeping 37463 word types
2017-05-13 11:04:07,335 : INFO : PROGRESS: at sentence #110000, processed 2457504 words, keeping 38901 word types
2017-05-13 11:04:07,375 : INFO : PROGRESS: at sentence #120000, processed 2679065 words, keeping 40246 word types
2017-05-13 11:04:07,416 : INFO : PROGRESS: at sentence #130000, processed 2905623 words, keep

In [33]:
for i in downsample:
    w2w_model_strict = word2vec.Word2Vec.load('../../Results/Drosophila/models/downsample/dros_strict_downsample_parameter_'+str(i[0])+'_model')
    w2w_model_gen = word2vec.Word2Vec.load('../../Results/Drosophila/models/downsample/dros_gen_downsample_parameter_'+str(i[0])+'_model')
    w2w_model_be = word2vec.Word2Vec.load('../../Results/Drosophila/models/downsample/dros_be_downsample_parameter_'+str(i[0])+'_model')
    
    for seed in random_seeds:
        data_name = '../../Results/Drosophila/train_val/dros_tr_val_split_' + str(seed)
        train_data = pickle.load(open(data_name + '_train_data.pkl', 'rb'))
        train_labels = pickle.load(open(data_name + '_train_labels.pkl', 'rb'))
        validation_data = pickle.load(open(data_name + '_test_data.pkl', 'rb'))
        validation_labels = pickle.load(open(data_name + '_test_labels.pkl', 'rb'))
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_strict, feature_count=600)
        
        strict_list_SR_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                     train_labels, validation_labels]
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_gen, feature_count=600)
        
        strict_list_GEN_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                      train_labels, validation_labels]
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_be, feature_count=600)
        
        strict_list_BE_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                     train_labels, validation_labels]
        
        pickle.dump(strict_list_SR_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/downsample/dros_strict_list_SR_downsample_param_'+str(i[0])+'_'+str(seed)+'_results_list.pkl', 'wb'))
        pickle.dump(strict_list_GEN_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/downsample/dros_strict_list_GEN_downsample_param_'+str(i[0])+'_'+str(seed)+'_results_list.pkl', 'wb'))
        pickle.dump(strict_list_BE_dims_param, open('../../Results/Drosophila/result_list/w2v_param_search/downsample/dros_strict_list_BE_downsample_param_'+str(i[0])+'_'+str(seed)+'_results_list.pkl', 'wb'))
        
        strict_final_list = [strict_list_SR_dims_param, 
                             strict_list_GEN_dims_param, 
                             strict_list_BE_dims_param]
        print ('\nPredicting\n')
        errors = []
        fpr = []
        tpr = []

        for entry in strict_final_list:
            error_w2v_norm, probs_w2v_norm = pred.XGB_classifier(entry[0], entry[1],
                                                                 entry[2], entry[3])
            fpr_w2v_norm, tpr_w2v_norm, _ = roc_curve(entry[3], probs_w2v_norm)
            error_w2v_fs, probs_w2v_fs = pred.XGB_classifier(entry[0], entry[1],
                                                             entry[2], entry[3],
                                                             feature_selection=True)
            fpr_w2v_fs, tpr_w2v_fs, _ = roc_curve(entry[3], probs_w2v_fs)

            errors.append([error_w2v_norm, error_w2v_fs])
            fpr.append([fpr_w2v_norm, fpr_w2v_fs])
            tpr.append([tpr_w2v_norm, tpr_w2v_fs])
            
        pickle.dump(errors, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/downsample/dros_downsample_param'+str(i[0])+'_errors_pickle_'+str(seed)+'.pkl', 'wb'))
        pickle.dump(fpr, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/downsample/dros_downsample_param'+str(i[0])+'_fpr_pickle_'+str(seed)+'.pkl', 'wb'))
        pickle.dump(tpr, open('../../Results/Drosophila/error_fpr_tpr/w2v_param_search/downsample/dros_downsample_param'+str(i[0])+'_tpr_pickle_'+str(seed)+'.pkl', 'wb'))

2017-05-13 11:04:19,786 : INFO : loading Word2Vec object from ../../Results/Drosophila/models/downsample/dros_strict_downsample_parameter_1_model
2017-05-13 11:04:20,574 : INFO : loading wv recursively from ../../Results/Drosophila/models/downsample/dros_strict_downsample_parameter_1_model.wv.* with mmap=None
2017-05-13 11:04:20,575 : INFO : setting ignored attribute syn0norm to None
2017-05-13 11:04:20,576 : INFO : setting ignored attribute cum_table to None
2017-05-13 11:04:20,577 : INFO : loaded ../../Results/Drosophila/models/downsample/dros_strict_downsample_parameter_1_model
2017-05-13 11:04:20,589 : INFO : loading Word2Vec object from ../../Results/Drosophila/models/downsample/dros_gen_downsample_parameter_1_model
2017-05-13 11:04:21,536 : INFO : loading wv recursively from ../../Results/Drosophila/models/downsample/dros_gen_downsample_parameter_1_model.wv.* with mmap=None
2017-05-13 11:04:21,537 : INFO : setting ignored attribute syn0norm to None
2017-05-13 11:04:21,538 : INFO 


Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)


...


Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


../Core-scripts/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


In [44]:
best_parameter = []
for seed in random_seeds:
    mean_err_dros_strict = []
    mean_auc_dros_strict = []
    mean_err_dros_gen = []
    mean_auc_dros_gen = []
    mean_err_dros_be = []
    mean_auc_dros_be = []
    for i in downsample:
        drct = '../../Results/Drosophila/error_fpr_tpr/w2v_param_search/downsample/'
        errors_dros = mult_open(drct, str(i[0])+'_errors_pickle_'+str(seed))
        fpr_dros = mult_open(drct, str(i[0])+'_fpr_pickle_'+str(seed))
        tpr_dros = mult_open(drct, str(i[0])+'_tpr_pickle_'+str(seed))
        for e, f, t in zip(errors_dros, fpr_dros, tpr_dros):
            input_list = [[e, f, t]]
            name_list = ['drosophila']
            for idx in range(3):
                for item, name in zip(input_list, name_list):
                    for fpr_item, tpr_item, error_item in zip(item[1][idx], item[2][idx], item[0][idx]):
                        roc_auc = auc(fpr_item, tpr_item)
                        auc_val = '%.3f' % roc_auc
                        error = '%.3f' % error_item
                        if idx == 0:
                            mean_err_dros_strict.append(error_item)
                            mean_auc_dros_strict.append(roc_auc)
                        elif idx == 1:
                            mean_err_dros_gen.append(error_item)
                            mean_auc_dros_gen.append(roc_auc)
                        elif idx == 2:
                            mean_err_dros_be.append(error_item)
                            mean_auc_dros_be.append(roc_auc)

    mean_err_dros_org_strict = mean_err_dros_strict[0::2]
    mean_err_dros_fs_strict = mean_err_dros_strict[1::2]
    mean_auc_dros_org_strict = mean_auc_dros_strict[0::2]
    mean_auc_dros_fs_strict = mean_auc_dros_strict[1::2]

    mean_err_dros_org_gen = mean_err_dros_gen[0::2]
    mean_err_dros_fs_gen = mean_err_dros_gen[1::2]
    mean_auc_dros_org_gen = mean_auc_dros_gen[0::2]
    mean_auc_dros_fs_gen = mean_auc_dros_gen[1::2]

    mean_err_dros_org_be = mean_err_dros_be[0::2]
    mean_err_dros_fs_be = mean_err_dros_be[1::2]
    mean_auc_dros_org_be = mean_auc_dros_be[0::2]
    mean_auc_dros_fs_be = mean_auc_dros_be[1::2]
    
    org_error = max([(mean_err_dros_org_strict.index(min(mean_err_dros_org_strict))+1), 
                     (mean_err_dros_org_gen.index(min(mean_err_dros_org_gen))+1), 
                     (mean_err_dros_org_be.index(min(mean_err_dros_org_be))+1)])
    
    fs_error = max([(mean_err_dros_fs_strict.index(min(mean_err_dros_fs_strict))+1), 
                     (mean_err_dros_fs_gen.index(min(mean_err_dros_fs_gen))+1), 
                     (mean_err_dros_fs_be.index(min(mean_err_dros_fs_be))+1)])
    
    org_auc = max([(mean_auc_dros_org_strict.index(max(mean_auc_dros_org_strict))+1), 
                     (mean_auc_dros_org_gen.index(max(mean_auc_dros_org_gen))+1), 
                     (mean_auc_dros_org_be.index(max(mean_auc_dros_org_be))+1)])
    
    fs_auc = max([(mean_auc_dros_fs_strict.index(max(mean_auc_dros_fs_strict))+1), 
                     (mean_auc_dros_fs_gen.index(max(mean_auc_dros_fs_gen))+1), 
                     (mean_auc_dros_fs_be.index(max(mean_auc_dros_fs_be))+1)])
    
    best_parameter.append((seed, [org_error, fs_error, org_auc, fs_auc]))

In [45]:
best_overall = []
for i in best_parameter:
    best_overall.append(most_common(i[1]))
    print(i)
best_downsample = most_common(best_overall)
print(best_downsample)

(144, [3, 5, 6, 5])
(235, [3, 6, 6, 5])
(905, [5, 5, 6, 6])
(2895, [5, 3, 5, 3])
(3462, [3, 5, 6, 6])
(4225, [2, 5, 5, 5])
(5056, [5, 6, 3, 3])
(5192, [4, 6, 5, 5])
(7751, [4, 5, 6, 5])
(7813, [6, 3, 6, 5])
5


In [43]:
best_w2v_parameters = [best_dims, best_word_count, 4, best_context_window, downsample[5][1]]
print(best_w2v_parameters)

[600, 6, 4, 6, 1e-06]


In [30]:
for i in downsample:
    print(i)


0.1
0.01
0.001
0.0001
1e-05
1e-06
